# 分组函数`groupby()`详解

本节详细介绍`groupby()`的使用。

In [ ]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 使用语法

`Series` 与 `DataFrame` 对象都支持`groupby()`功能，其使用语法为：
```python
groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs)
```
主要参数
- `by=None`,映射，函数，标签或标签列表。
- `axis=0`, 整数。表示操作的轴向，0表示列，1表示行。
- `level=None`, 整数或索引名，指定层级索引。
- `as_index=True`，布尔数。表示聚合后的标签是否为`DataFrame`索引形式。设置为False，则使用“SQL风格”。
- `sort=True`, 布尔数。是否对分组列或标签进行排序。
- `group_keys=True`，布尔数。是否显示分组标签的名称。
- `squeeze=False`，布尔数。是否在可行情况下对返回数据进行降维。
- `observed=False`，布尔数。针对类别数据。

## 指定列或数组

下面创建一个`Series`对象与`DataFrame`对象：

In [ ]:
ser = pd.Series(range(6))
ser

In [ ]:
keys = ['A', 'B', 'C', 'A', 'B', 'C']
df = pd.DataFrame({'key': keys, 'data': range(6)})
df

下面分别调用`Series`与`DataFrame`对象的`groupby()`方法：

In [ ]:
# 使用keys进行分组
g1 = ser.groupby(keys)
# 使用key列进行分组
g2 = df.groupby('key')
# 返回类型
type(g1), type(g2)

二者都返回分组（`GroupBy`）对象，分别是`SeriesGroupBy`与`DataFrameGroupBy`的实例对象。`GroupBy`对象支持迭代，会生成各个分组：

In [ ]:
for key, group in g1:
    print(key, type(group))
    print(group)

In [ ]:
for key, group in g2:
    print(key, type(group))
    print(group)

有上可见，`groupby()`只是进行分组，对于`Series`对象其分组数据为序列，`DataFrame`对象的分组数据是`DataFrame`对象。要进行后续的应用与联合操作，可以调用`GroupBy`对象的各种方法，下表列出常用描述性统计方法：
- `count()`，非NA值计数。
- `min(), max()`，非NA值的最小值与最大值。
- `sum()`，非NA值求和。
- `mean(), median()`，非NA值均值与中位数。
- `std(), var()`，非NA值标准差与方差。
- `first(), last()`，非NA值第一个与最后一个。
- `size()`，大小
- `prod`，非NA值的乘积

下面调用`GroupBy`对象的`max()`方法，会对各个分组数据应用`max()`函数，并把结果合并起来：

In [ ]:
g1.max()

In [ ]:
g2.max()

## 指定多列

在使用`groupby()`对`DataFrame`对象进行分组时，可以指定`DataFrame`多个列。下面创建一个`DataFrame`对象：

In [ ]:
dict_obj = {'key1' : ['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      np.nan, 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8),
            'data3': ['1', '2', '3', '4', '5', '6', '7', '8']}
df = pd.DataFrame(dict_obj)
df

指定多列来进行分组：

In [ ]:
g3 = df.groupby(['key1', 'key2'])
[key for key, group in g3]

可以看出此时`GroupBy`对象的键值为元组。接着应用`mean()`方法，对各个分组数据进行求均值并合并结果：

In [ ]:
# 丢掉NA值行
df.groupby(['key1', 'key2']).mean()

在处理过程中，NA值行会被丢掉。缺省情况下，会对所有列进行处理，由于`mean()`主要用于数值型故会忽略`data3`列（字符串类型）。使用`GroupBy`对象的运算符`[]`可以处理指定列：

In [ ]:
# 分组计数
df.groupby(['key1', 'key2'])['data1', 'data3'].count()

## 使用字典

除了指定列或数组外，还可以使用映射方式。例如下面对象为不同人不同语言考试成绩：

In [ ]:
scores = pd.DataFrame(np.random.randint(50, 90, (5,5)),
                      columns=['Python', 'R', 'C', 'C++', 'Java'],
                      index=['张三', '李四', '王五', '赵东', '钱西'])
scores

已知列的分组关系，使用如下映射:

In [ ]:
mapping = {'Python': '脚本', 'R': '脚本', 'C': '编译',
           'C++': '编译', 'Java':'编译', 'Matlab':'编译'}

下面进行分组计算，得到脚本语言与编译语言的平均成绩：

In [ ]:
scores.groupby(mapping, axis=1).mean()